In [1]:
import pandas as pd
import numpy as np
import tweepy
import requests
import requests_cache
import time
import sqlite3 as sql
requests_cache.install_cache('demo_cache')

Types of accounts that MIB Dataset provides:

genuine_accounts: verified accounts that are human-operated

social_spambots_1: retweeters of an Italian political candidate

social_spambots_2: spammers of paid apps for mobile devices

social_spambots_3: spammers of products on sale at Amazon.com

traditional_spambots_1: training set of spammers used by C. Yang, R. Harkreader, and G. Gu.

traditional_spambots_2: spammers of scam URLs

traditional_spambots_3: automated accounts spamming job offers

traditional_spambots_4: another group of automated accounts spamming job offers

fake_followers: simple accounts that inflate the number of followers of another account

TFP: (the fake project): 100% humans

E13: (elections 2013): 100% humans

INT: (intertwitter): 100% fake followers

FSF: (fastfollowerz): 100% fake followers

TWT: (twittertechnology): 100% fake followers

In [2]:
# Twitter API setup
  
CONSUMER_KEY = 'QifrSmYAQ2mjIf8kiPoL2kI4v'
CONSUMER_SECRET = 'qHXPmXDd4Gw2cqZ5zKmDpU6drKEHTF396pj9qyoUYWDcLsFOlm'
ACCESS_TOKEN = '3379234805-hUmhUJa0oV9V1mnKDuB6bitJ1QTEjdq2c9zE0RA'
ACCESS_TOKEN_SECRET = 'cQiqDWDuoLj8SH68d6JuPhthToImk3WzmcQ3pbyxYfYK1'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [3]:
def get_user_info(twitter_id, col_names):
    """
    creates a dataframe that stores all the user information
    
    Arguments: a list of twitter_id
    
    Return: Dataframe
    """
    info = []
    status = []
    
    for twitter in twitter_id:
        try:
            user = api.get_user(twitter)
            get_var = [user.id, user.name, user.screen_name, user.statuses_count, user.followers_count, user.friends_count, user.favourites_count,
                        user.listed_count, user.url, user.lang, user.time_zone, user.location, user.default_profile, user.default_profile_image,
                        user.geo_enabled, user.profile_image_url, user.profile_use_background_image, user.profile_background_image_url_https,
                        user.profile_text_color, user.profile_image_url_https, user.profile_sidebar_border_color, user.profile_background_tile,
                        user.profile_sidebar_fill_color, user.profile_background_image_url, user.profile_background_color, user.profile_link_color,
                        user.utc_offset, user.is_translator,user.follow_request_sent, user.protected, user.verified, user.notifications,
                        user.description, user.contributors_enabled, user.following, user.created_at]
        except tweepy.TweepError as error:
            status.append(error[0][0]['message'])
            
            if error[0][0]['message'] == 'Rate limit exceeded':
                        print 'Rate limited. Sleeping for 15 minutes.'
                        time.sleep(15 * 60 + 15)
                        continue

        info.append(get_var)
        
    # create the dataframe
    data = pd.DataFrame(info)
    data.columns = col_names
    
    # find the existing twitter account information
    check = data['id'] == twitter_id
    final_data = data[check]
    final_data.rename(columns = {'name': 'user_name'}, inplace=True)
    return final_data, status


In [8]:
def create_user_df(account_type):
    """
    get the twitter id from the csv files and create a dataframe that stores all the user information
    
    Argument: account_type, col_names
    
    Return: dataframe and a list of status
    """
    df = pd.read_csv(account_type + ".csv/users.csv")
    # get the twitter ids from the datasets
    twitter_id = list(df['id'])
    names = list(df.columns.values)
    # remove uncommon columns
    col_names = list(df.columns.values)[:-5]
    # remove the profile_banner_url
    col_names.remove('profile_banner_url')
    final_df, status = get_user_info(twitter_id, col_names)
    final_df.name = account_type
    return final_df, status

In [9]:
def get_tweets_limit(id_list):
    
    """
    gets the most recent 200 tweets from a user timeline
    
    Argument: a list of twitter id
    
    Return: a tweets dataframe
    """
    # create a list that stores the tweets for all the users
    all_tweets = []
    for twitter_id in id_list:
        try:
            # read the tweets from the timeline, 200 is the maximum allowed count
            tweets = api.user_timeline(user_id = twitter_id,count = 200)
             # save the tweets into the list
            tweets_list = []
            tweets_list.extend(tweets)
            outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in tweets_list]
            tweets = [tweets.append(twitter_id) for tweets in outtweets]
        except tweepy.TweepError as error:
            pass
        
        all_tweets.extend(outtweets)
        
    tweets_df = pd.DataFrame(all_tweets) 
    col_names = ['tweet_id', 'tweet_created_at', 'tweet_text', 'twitter_id']
    tweets_df.columns = col_names
    #tweets_df['twitter_id'] = twitter_id
    return tweets_df

In [10]:
def create_tweets_df(user_df):
    """
    create a dataframe that stores the existing user tweets
    
    Argument: the user info dataframe
    
    Return: a tweets dataframe
    """
    
    twitter_id = list(user_df['id'])
    tweets_df = get_tweets_limit(twitter_id)
    return tweets_df

In [11]:
def create_table_user(account_type):
    """
    create tables for twitter user information
    
    Argument: account_type
    
    Return: tables in the database
    """
    df_user, status = create_user_df(account_type)
    conn = sql.connect("twitter.sqlite")
    df_table_user = df_user.to_sql(account_type + "_users_info", conn, if_exists = 'replace', index = False)
    return df_table_user


In [17]:
def create_table_tweets(user_df):
    """
    create tables for different twitter accounts
    
    Argument: account_type
    
    Return: tables in the database
    """
    df_tweets = create_tweets_df(user_df)
    conn = sql.connect("twitter.sqlite")
    conn.text_factory = str
    user_df_name = user_df.name
    df_table_tweets = df_tweets.to_sql(user_df_name + "_tweets", conn, if_exists = 'replace', index = False)
    return df_table_tweets

In [18]:
test_df, status = create_user_df('test')

In [19]:
create_table_user('test')

In [20]:
create_table_tweets(test_df)

In [21]:
conn = sql.connect("twitter.sqlite")
pd.read_sql("SELECT * FROM test_users_info", conn)

,id,user_name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,utc_offset,is_translator,follow_request_sent,protected,verified,notifications,description,contributors_enabled,following,created_at
0,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,NaN,0,0,0,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35
1,2492782375,ローラー,1120Roll,6180,350,554,5542,5,None,ja,...,32400.0,0,0,0,0,0,ゴロゴロするのが大好きなちなDeです。政治思想は掻い摘み派です。メモ帳化中。無言フォロー失礼...,0,0,2014-05-13 10:37:57
2,293212315,Kelsey Brown,keke_brooke,1874,241,197,2247,1,None,en,...,-14400.0,0,0,0,0,0,Future mall Santa,0,0,2011-05-04 23:30:37
3,191839658,bio,pinkfunkys,262963,2164,946,87959,93,None,en,...,-7200.0,0,0,0,0,0,no longer here I'm @toujoursbeiles,0,0,2010-09-17 14:02:10
4,3020965143,Ms Kathy,191a5bd05da04dc,140,23,96,19,0,None,en,...,NaN,0,0,0,0,0,Cosmetologist,0,0,2015-02-06 04:10:49
5,2157382005,JO🎶🎹🎶,19_Joanne_87,39516,1812,1290,2359,23,https://t.co/Hj1KqfMNj6,en,...,-14400.0,0,0,0,0,0,Communications Degree; Graphic Designer; makeu...,0,0,2013-10-28 16:08:18
6,1947320929,❄McKayla❄,1Dniallprincess,15737,905,1939,21283,8,None,en,...,NaN,0,0,0,0,0,"Live, Young, Wild and Free #crazymofo",0,0,2013-10-08 15:59:30
7,255846106,Giselle,1GisellePizarro,83085,443,98,3020,4,https://t.co/uB4DXntmZt,es,...,-10800.0,0,0,0,0,0,A 23-year-old #Rusher. I used to write a lot. ...,0,0,2011-02-22 04:37:34


In [23]:
pd.read_sql("SELECT * FROM test_tweets", conn)

,tweet_id,tweet_created_at,tweet_text,twitter_id
0,853218018830065665,2017-04-15 12:06:42,RT @LOL_Fxxkle0un: 続 揃うとダッサいブランド、supreme。 http...,1502026416
1,853144976376553472,2017-04-15 07:16:27,@KaiOyazhg わろた,1502026416
2,853048796346933248,2017-04-15 00:54:16,@touya1432 @yuto45436620 おめでとう！🎉また走り行こうね！,1502026416
3,850774922112544768,2017-04-08 18:18:42,結局三時間睡眠だふぁっく,1502026416
4,850569793824374785,2017-04-08 04:43:36,ひさびささんずうぃずしもぞの https://t.co/lPSnEFyeaj,1502026416
5,850161030705500160,2017-04-07 01:39:19,@kaitnn0602 ええじゃん,1502026416
6,850097177753812992,2017-04-06 21:25:35,@kaitnn0602 こいよ‼️,1502026416
7,849967812550447104,2017-04-06 12:51:32,@kaitnn0602 なんぞや,1502026416
8,849368222990204931,2017-04-04 21:08:59,なんだかんだ免許取って一年かー\n２回事故って死にかけて、バイク一台廃車にしたけど、行きてる...,1502026416
9,849153469663424512,2017-04-04 06:55:38,@115117masaya たまには遊ぼうぜ！,1502026416


In [27]:
# Join two databases together
pd.read_sql("SELECT * FROM test_users_info INNER JOIN test_tweets ON test_users_info.id = test_tweets.twitter_id", conn)

,id,user_name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,verified,notifications,description,contributors_enabled,following,created_at,tweet_id,tweet_created_at,tweet_text,twitter_id
0,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,806468777458036736,2016-12-07 12:01:54,@kaitnn0602 しかとはだめだないくら、かいとが不細工なくせに彼女がいたとしても笑,1502026416
1,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,806470120344195072,2016-12-07 12:07:14,@kaitnn0602 かなしい,1502026416
2,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,806470336367558657,2016-12-07 12:08:06,@kaitnn0602 ジョークを言い合える仲じゃないか。,1502026416
3,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,806470559336787968,2016-12-07 12:08:59,@kaitnn0602 そうね。そーりー,1502026416
4,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,806470627490004994,2016-12-07 12:09:15,とりあえずおれにクリスマスプレゼントちょうだい,1502026416
5,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,806472381849956353,2016-12-07 12:16:14,@kaitnn0602 ありがと。てんくん。\nせめて🍦こっち。でしょ,1502026416
6,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,807400206685900800,2016-12-10 01:43:04,New#cap#68andbrothers#68bros#red https://t.co/...,1502026416
7,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,808237217717944321,2016-12-12 09:09:03,@Yuuta88958635 やろ！,1502026416
8,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,808268367505199106,2016-12-12 11:12:50,足にクレーターができた,1502026416
9,1502026416,早川援,0918Bask,3431,286,441,586,0,None,ja,...,0,0,motorcycle.&car.,0,0,2013-06-11 11:20:35,808963756638269440,2016-12-14 09:16:04,マジイライラする死ね,1502026416


In [ ]:
# close the database
conn.close()